### CNN 학습 main 코드

### 1. jpg to numpy

In [1]:
import glob
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

class_num = 5

train_files = glob.glob("5_dataset/train/*/*.jpg")
val_files = glob.glob("5_dataset/val/*/*.jpg")
test_files = glob.glob("5_dataset/test/*/*.jpg")

y_temp_train = []
y_temp_validation = []
y_temp_test = []

train_temp_files = []
val_temp_files = []
test_temp_files = []

for _file in train_files:
    train_temp_files.append(_file)
    _label = _file.split("\\")
    y_temp_train.append(int(_label[1][4]))

for _file in val_files:
    val_temp_files.append(_file)
    _label = _file.split("\\")
    y_temp_validation.append(int(_label[1][4]))    

for _file in test_files:
    test_temp_files.append(_file)
    _label = _file.split("\\")
    y_temp_test.append(int(_label[1][4]))  
    
image_width = 331
image_height = 221
channels = 3

x_train = np.ndarray(shape = (len(train_temp_files), image_width, image_height, channels), dtype = np.float32)
y_train = np.ndarray(shape = (len(y_temp_train), class_num), dtype = np.float32)
x_validation = np.ndarray(shape = (len(val_temp_files), image_width, image_height, channels), dtype = np.float32)
y_validation = np.ndarray(shape = (len(y_temp_validation), class_num), dtype = np.float32)
x_test = np.ndarray(shape = (len(test_temp_files), image_width, image_height, channels), dtype = np.float32)
y_test = np.ndarray(shape = (len(y_temp_test), class_num), dtype = np.float32)

Using TensorFlow backend.


In [4]:
def one_hot_encoding(num):
    arr = [0, 0, 0, 0, 0]
    for i in range(class_num):
        if (i==num):
            arr[i] = 1
        else:
            arr[i] = 0
    return arr

In [5]:
i = 0
for _file in train_files:
    img = load_img(_file)
    img.thumbnail((image_width, image_height))
    x = img_to_array(img)
    x = x.reshape((331, 221, 3))
    x = (x-128.0)/128.0
    x_train[i] = x
    y_train[i] = one_hot_encoding(y_temp_train[i])
    i += 1
print("Train data converting complete")

i = 0
for _file in val_files:
    img = load_img(_file)
    img.thumbnail((image_width, image_height))
    x = img_to_array(img)
    x = x.reshape((331, 221, 3))
    x = (x-128.0)/128.0
    x_validation[i] = x
    y_validation[i] = one_hot_encoding(y_temp_validation[i])
    i += 1
print("validation data converting complete")

i = 0
for _file in test_files:
    img = load_img(_file)
    img.thumbnail((image_width, image_height))
    x = img_to_array(img)
    x = x.reshape((331, 221, 3))
    x = (x-128.0)/128.0
    x_test[i] = x
    y_test[i] = one_hot_encoding(y_temp_test[i])
    i += 1
print("test data converting complete")

Train data converting complete
validation data converting complete
test data converting complete


In [ ]:
print("x_train : " + str(type(x_train)))
print("y_train : " + str(type(y_train)))
print("x_validation : " + str(type(x_validation)))
print("y_validation : " + str(type(y_validation)))
print("x_test : " + str(type(x_test)))
print("y_test : " + str(type(y_test)))

---

### 2. CNN with keras
* ref url
    * https://datascienceschool.net/view-notebook/51e147088d474fe1bf32e394394eaea7/
    * http://pythonstudy.xyz/python/article/402-numpy-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0
    * https://snowdeer.github.io/machine-learning/2018/01/10/convolution-neural-network/

In [ ]:
from keras import layers
from keras import models
from keras import optimizers
import keras

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.regularizers import l1, l2

import matplotlib.pyplot as plt
import os
import numpy

#### <사용한 model 정보>
* model1 : CRM CRM CRM CRM CRM FDD, epoch = 20, batch_size = 50
* model2 : CRM CRM CRM CRM CRM FDD, epoch = 15, batch_size = 60
* model3 : CRM CRM CRM CRM CRM FDD, epoch = 15, batch_size = 50 - good
* model4 : CRM CRM CRM CRM CRM Drop(0.5) FDD, epoch = 15, batch_size = 50
* model5 : CRM CRM Drop(0.25) CRM CRM CRM  FDD, epoch = 15, batch_size = 50
* model6 : CRM CRM CRM CRM CRM Drop(0.25) FDD, epoch = 15, batch_size = 50
---
* 이 밑의 모델들은 비교적 후지다.  사용 안할 것
    + model : CRM CRM + L2reg(0.001) CRM CRM CRM FDD, epoch = 15, batch_size = 50
    + model : CRM CRM + L2reg(0.001) CRM CRM CRM Dropout(0.5) FDD, epoch = 15, batch_size = 50 
    + model : CRM CRM CRM CRM CRM FDD, epoch = 15, batch_size = 40
    + model : CRM CRM CRM Drop(0.25) CRM CRM CRM FDD, epoch = 15, batch_size = 50

In [ ]:
model_num = 7

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(331, 221, 3))) # input size : 331, 221, 3
model.add(layers.MaxPool2D((2,2)))

# model.add(layers.Conv2D(64, (3, 3), activity_regularizer = l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))

model.add(Dropout(0.25))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(class_num, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy',
             optimizer = optimizers.RMSprop(lr = 1e-4),
             metrics = ['acc'])

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=15, batch_size=50, 
                    validation_data=(x_validation, y_validation),
                    verbose=1)

In [ ]:
model.save('5classSoundModel'+str(model_num)+'.h5')

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label = 'Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation loss')
plt.legend()
plt.savefig("5classSoundModelGraph"+str(model_num)+"-1.png")

plt.figure()

plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.savefig("5classSoundModelGraph"+str(model_num)+"-2.png")

plt.show()

---

### 3. 실제 test set에 대해 잘못된 판별 갯수 파악

In [ ]:
import glob
from tqdm import tqdm_notebook
testList = glob.glob("5_dataset/test/*/*.jpg")
total = [0, 0, 0, 0, 0, 0]
result = [0, 0, 0, 0, 0, 0]
print("Length : " + str(len(testList)))
for i in tqdm_notebook(range(len(testList))):
    predictNum = model.predict_classes(x_test[i:i+1, :], verbose = 0)
    _ArrRealNum = testList[i].split("\\")
    realNum = int(_ArrRealNum[1].split("fold")[-1])
    total[realNum] = total[realNum] + 1
    if(predictNum != realNum):
        result[realNum] = result[realNum] + 1
print("total : " , end = "")
print(total)
print("result : " , end = "")
print(result)
print("error% : ", end = "")
for i in range(6):
    temp = result[i] / total[i]
    temp = temp * 100
    print(round(temp, 3), end = ", ")

In [ ]:
num = [0, 0, 0, 0, 0, 0]
for i in y_test:
    for j in range(6):
        if(int(i[j]) == 1):
            num[j] += 1
for i in range(1, 6):
    num[i] = num[i] + num[i-1]
print("0 to " + str(num[0]))
print(str(num[0]+1) + " to " + str(num[1]))
print(str(num[1]+1) + " to " + str(num[2]))
print(str(num[2]+1) + " to " + str(num[3]))
print(str(num[3]+1) + " to " + str(num[4]))
print(str(num[4]+1) + " to " + str(num[5]))

In [ ]:
num = 0
for i in range(len(testList)):
    print(str(num), end = " : ")
    print(model.predict_classes(x_test[i:i+1, :], verbose = 0), end = " , ")
    if(i%10==0 and i != 0):
        print("\n")
    if(i==86 or i == 285 or i == 484 or i == 559 or i == 745):
        print("\n===========================================================================")
        print("\n")
    num += 1